**readme**: scripts below are used to sample certain amount of unlabeled data (5000) and tokenized text

In [128]:
import pandas as pd
import os
import re
import csv
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import jieba
import jieba.posseg as pseg
import jieba.analyse

import glob
os.chdir('/Users/liyuan/desktop/CSAir')

In [133]:
class Tokenization():
    def __init__(self, input_data, output_name, stopwords):
        self.input = input_data[:]
        self.output_name = str(output_name)
        self.sentences = []
        self.tfidf_score = []
        self.stopwords = stopwords
    
    def get_tokenized_sents(self):        
        for sent in self.input:
            tokenized_sent = ' '.join(word for word in jieba.cut(sent.strip()) if word not in self.stopwords)
            # remove digits
            tokenized_sent = re.sub(r'\d+','',tokenized_sent)
            # remove punctuation
            tokenized_sent = re.sub(r'[^\w\s]','', tokenized_sent)
            # remove non-chinese characters
            # match all Chinese words
            re_words = re.compile(u"[\u4e00-\u9fa5]+")
            res = re.findall(re_words, tokenized_sent)
            if res:
                valid_tokenized_sent = ' '.join([r for r in res])
            self.sentences.append(valid_tokenized_sent)
        # tokenized output to txt file        
        with open(self.output_name + '.txt','w',newline='') as output_file:
            for line in self.sentences:
                output_file.write(line + '\n')
        return self.sentences
    
    def get_topN_tf_idf(self, content, topK=20):
        tags = jieba.analyse.extract_tags(content, topK)
        return " ".join(tags)

In [87]:
import codecs
# load data: 2018年季度APP反馈数据分析/反馈数据2018年6-7-8月数据
F = codecs.open('Source_Data/第二批data/2018年季度APP反馈数据分析/反馈数据2018年6-7-8月数据.csv','r','utf-8')
input_data =F.readlines()[2:]
F.close()

# get list of reviews
fk_review_list_678 = []
for i in range(len(input_data)):
    try:
        review = input_data[i].split(',')[9]
        fk_review_list_678.append(review)
    except:
        continue
print(len(fk_review_list_678))

# load data: 2018年季度APP反馈数据分析/反馈数据3-5月原始数据.csv
F = codecs.open('Source_Data/第二批data/2018年季度APP反馈数据分析/反馈数据3-5月原始数据.csv','r','utf-8')
input_data =F.readlines()[1:]
F.close()

# get list of reviews
fk_review_list_345 = []
for i in range(len(input_data)):
    try:
        review = input_data[i].split(',')[7]
        fk_review_list_345.append(review)
    except:
        continue
print(len(fk_review_list_345))

# load data: 2018年季度APP反馈数据分析/反馈数据3-5月原始数据.csv
F = codecs.open('Source_Data/第二批data/2018年季度APP反馈数据分析/反馈数据9-10-11.csv','r','utf-8')
input_data =F.readlines()[2:]
F.close()

# get list of reviews
fk_review_list_91011 = []
for i in range(len(input_data)):
    try:
        review = input_data[i].split(',')[12]
        fk_review_list_91011.append(review)
    except:
        continue
print(len(fk_review_list_91011))

# concat unlabeled reviews into one file
fk_reviews = []
fk_reviews += fk_review_list_345
fk_reviews += fk_review_list_678
fk_reviews += fk_review_list_91011
print('反馈数据 in total has %d reviews'% len(fk_reviews))
print('反馈数据举3例：', fk_reviews[:3])

8233
8776
7224
反馈数据 in total has 24233 reviews
反馈数据举3例： ['今天的CZ3539航班的毛毯质量很差，下来发现裤子上掉落很多毛。', '最近经常出现提前很久打电话预留前排靠过道位置，到机场就没有，一直道歉，作为高端客户，感觉很不好。', '最近航班服务经常出问题，提前很久打电话预留前排靠过道位置，到机场就没有物流，作为高端客户，对这样的服务很不满意。']


In [107]:
# sample 5000 user reviews from fk_reivews; 
# need to keep the index of each sampled reviews; ultimately would need manual evualuation
import random
sample_size = 5000 
# get the index list of the sampled reviews
sample_index_list = [i for i in random.sample(range(len(fk_reviews)),sample_size)]
sample_reviews = [fk_reviews[i] for i in random.sample(range(len(fk_reviews)), sample_size)]
print(len(sample_reviews))

5000


In [134]:
stopwords = [line.strip() for line in open('Source_Data/stopwords.txt', 'r', encoding='utf-8').readlines()]   
tok = Tokenization(sample_reviews,'res/fk_reviews_tokenized',stopwords)
sampled_reviews_tokenized = tok.get_tokenized_sents()

In [132]:
# TODO: need to combine train and test words; and create a new countvector
unlabeled_reviews = sampled_reviews_tokenized
unlabeled_reviews = np.array(unlabeled_reviews)
unlabeled_reviews

array([['发餐 时 牛肉 饭 成本 原因'],
       ['收到 信息 买 机票 短信'],
       ['旅行 网上 买 一张 乌鲁木齐 机票 身份证 填写 士兵证 港澳台 证 一栏 填写 付款 明天 取 机票 取出'],
       ...,
       ['购买 二零一 八年 四月八 日 早上 九点 广州 飞 回 墨尔本 商务 舱 机票 年 月 日 晚上 南宁 飞 达 广州 广州 中转 住 一夜 南航 官网 选择 级 中转 免费 豪华酒店 机场 铂尔曼 酒店 以往 乘坐 南航 商务 舱 广州 中转 时 入住 铂尔曼 酒店 选择 晕车 厉害 查看 铂尔曼 选项 之内 请 心愿 谢谢 客票 号'],
       ['飞机 改成 飞机 提前 通知 要升 南航 明珠 会员 里 认证 感觉 超 不好'],
       ['钱包 异常 开通 闪付']], dtype='<U214')

In [126]:
all_df = pd.read_csv('res/all_labeled_data.csv')
all_df['review_tokens'].values

array([' 11 月 15 日 提前 预订 2018 年 11 月 27 日 长沙 飞往 沈阳 cz3983 航班 做好 相关 会议 安排 11 月 17 日 收到 航班 延误 推迟 下午 13 40 CZ6408 航班 只好 解释 调整 会议 时间 12 点 飞机场 通知 时间 调整 下午 四点 二十 晚上 六点 起飞 飞机场 整整 六个 小时 一整天 做 事 一再 失信 生意 伙伴 信任 失望 ',
       '航班 延误 登机口 升舱 活动 以原 航班 起飞时间 为准 办理 理解 ',
       '重庆 乌鲁木齐 南航 航班 天气 原因 延误 和田 乘坐 天津 航班 ', ...,
       '未 常规 用户 流程 思路 设计 开发者 思维 使用者 思维 体验 差 补打 电子 票 麻烦 内容 刷新 太慢 不合理 人性 地方 建议 好好 修改 太 符合 南韩 形象 ',
       '11 月 19 日订 太原 深圳 CZ3584 航班 11 月 23 日接 南航 客服 电话 称该 航班 取消 建议 改退 协调 改至 ZH9132 航班 29 日 太原 机场 安检 后侯机 机场 称该 航班 延误 询问 机场 值班人员 称 飞机 未至 备降 重庆 万州 不知 何时能 飞 值班人员 协助 查询 合适 航班 改签 机场 21 号 柜台 办理 退票 手续 称 机场 负责 取消 登机 手续 退款 需至 购票 办理 南航 APP 办理 退票 原价 580 元 机票 只退 100 元 南方 航空 深圳 航空 违约 未 深圳 参加 商务活动 申请 航班 延误 费 损失费 如未 收到 反馈 寄 律师函 走 法律 程序 ',
       '刷新 行程 显示 服务器 繁忙 '], dtype=object)

In [ ]:
# TODO: combine labeled and unlabeled and make prediction
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer 

# use countVectorizer for one-hot encoding
count_v0= CountVectorizer();  
counts_all = count_v0.fit_transform(all_df['review_tokens'])
count_v1= CountVectorizer(vocabulary=count_v0.vocabulary_)  
 
counts_train = count_v1.fit_transform(train.review_tokens)
print ("the shape of train is "+repr(counts_train.shape))

count_v2 = CountVectorizer(vocabulary=count_v0.vocabulary_)
counts_test = count_v2.fit_transform(test.review_tokens)
print ("the shape of test is "+repr(counts_test.shape))

# implement tf-idf
tfidftransformer = TfidfTransformer()
train_data = tfidftransformer.fit(counts_train).transform(counts_train)
test_data = tfidftransformer.fit(counts_test).transform(counts_test)

print(train_data.shape)
print(test_data.shape)

X_train = train_data
y_train = train.label_encoded
X_test = test_data
y_test = test.label_encoded

def get_precision(y_pred, y_test):
    '''this function returns a precision score for the model'''
    num = 0
    y_pred = y_pred.tolist()
    for i,pred in enumerate(y_pred):
        if int(pred) == int(y_test.values[i]):
            num += 1
    precision = float(num) / len(y_pred)
    print('precision: '+'{:.2f}'.format(precision))
    return precision